# Loggers in Union components
It is possible to record absorption that occurs within a Union system by using absorption loggers. There are several different absorption loggers that behave in similar ways. In this exercise you will get an instrument with a union system and will add absorption loggers to see where absorption happens in the system and to build a detector.

In [ ]:
import mcstasscript as ms
import load_quiz

In [ ]:
instrument = ms.McStas_instr("exercise_4", input_path="run_folder")
instrument.settings(output_path="data_folder/exercise_4")

instrument.add_component("init", "Union_init") # Necessary for current version of Union in McStas 3.X

quiz = load_quiz.exercise_5()

In [ ]:
# Set up Al material with incoherent and powder
Al_incoherent = instrument.add_component("Al_incoherent", "Incoherent_process")
Al_incoherent.sigma = "4*0.0082"
Al_incoherent.packing_factor = 1
Al_incoherent.unit_cell_volume = 66.4

Al_powder = instrument.add_component("Al_powder", "Powder_process")
Al_powder.reflections = "\"Al.laz\""

Al = instrument.add_component("Al", "Union_make_material")
Al.process_string = '"Al_incoherent,Al_powder"'
Al.my_absorption = "100*4*0.231/66.4"

# Set up YBaCuO with incoherent and single crystal
YBaCuO_incoherent = instrument.add_component("YBaCuO_incoherent", "Incoherent_process")
YBaCuO_incoherent.sigma = 2.105
YBaCuO_incoherent.unit_cell_volume = 173.28

YBaCuO_crystal = instrument.add_component("YBaCuO_crystal", "Single_crystal_process")
YBaCuO_crystal.set_parameters(
{"ax" : 3.816, "ay" : 0, "az" : 0,
 "bx" : 0, "by" : 3.886, "bz" : 0,
 "cx" : 0, "cy" : 0, "cz" : 11.677,
 "delta_d_d" : 5E-4, "mosaic" : 30, "barns" : 1,
 "reflections" : '"YBaCuO.lau"'})

YBaCuO = instrument.add_component("YBaCuO", "Union_make_material")
YBaCuO.process_string = '"YBaCuO_incoherent,YBaCuO_crystal"'
YBaCuO.my_absorption = 100*14.82/173.28

src = instrument.add_component("source", "Source_div")
src.xwidth = 0.01
src.yheight = 0.035
src.focus_aw = 0.01
src.focus_ah = 0.01
src.flux = 1E13
instrument.add_parameter("double", "wavelength", value=4.0, comment="[AA]  Mean wavelength of neutrons")
instrument.add_parameter("double", "d_wavelength", value=0.001, comment="[AA]  Wavelength spread of neutrons")
src.lambda0 = "wavelength"
src.dlambda = "d_wavelength"


# At a reference point to build the cryostat around
cryostat_center = instrument.add_component("cryostat_center", "Arm")
cryostat_center.set_AT([0, 0, 1], RELATIVE=src)

# Parameter for controlling sample rotation
A3_angle = instrument.add_parameter("A3_angle", value=30.99)

sample = instrument.add_component("sample", "Union_box")
sample.set_AT([0, 0, 0], RELATIVE=cryostat_center)
sample.set_ROTATED([0, A3_angle, 0], RELATIVE=cryostat_center)
sample.xwidth = 0.012
sample.yheight = 0.032
sample.zdepth = 0.009
sample.material_string = '"YBaCuO"'
sample.priority = 200

# Adding cryostat using the cryostat tool
orange_cryostat = ms.Cryostat("orange", instrument, max_priority=100)
orange_cryostat.set_AT(0, cryostat_center)

orange_cryostat.add_layer(inner_radius=70E-3/2, outer_radius=75E-3/2,
                          origin_to_top=160-3, top_thickness=-1E-3,
                          origin_to_bottom=40E-3, bottom_thickness=5E-3,
                          p_interact=0.2)

orange_cryostat.add_layer(inner_radius=120E-3/2, outer_radius=125E-3/2,
                          origin_to_top=190E-3, top_thickness=22E-3,
                          origin_to_bottom=80E-3, bottom_thickness=31E-3,
                          p_interact=0.2)

orange_cryostat.build(include_master=True)
master = instrument.get_component("orange_master") # grab master object

instrument.add_declare_var("double", "t_min")
instrument.add_declare_var("double", "t_max")

instrument.append_initialize("t_min = (wavelength - d_wavelength)*(1.0 - 0.18 + 0.5)/(K2V*2*PI);")
instrument.append_initialize("t_max = (wavelength + d_wavelength)*(1.0 + 0.28 + 0.5)/(K2V*2*PI);")
instrument.add_declare_var("char", "options", array=256)
instrument.append_initialize('sprintf(options,"banana, theta limits=[-180,180] bins=180, t limits=[%g %g] bins=300",t_min,t_max);')

banana_detector = instrument.add_component("banana_tof_detector", "Monitor_nD")
banana_detector.set_RELATIVE(cryostat_center)
banana_detector.xwidth = 1
banana_detector.yheight = 0.2
banana_detector.restore_neutron = 1
options = "options"
banana_detector.options = options
banana_detector.filename = '"tof_b.dat"'

banana_detector = instrument.add_component("banana_detector", "Monitor_nD")
banana_detector.set_RELATIVE(cryostat_center)
banana_detector.set_parameters(xwidth = 1.0, yheight = 0.2, filename='"banana.dat"', restore_neutron=1,
                               options='"banana, theta limits=[-180,180] bins=180"')

L_monitor = instrument.add_component("L_monitor", "L_monitor")
L_monitor.set_AT(0.3, RELATIVE=cryostat_center)
L_monitor.set_parameters(xwidth=0.3, yheight=0.3,
                         Lmin="wavelength - d_wavelength",
                         Lmax="wavelength + d_wavelength",
                         nL=300, filename='"l_mon.dat"', restore_neutron=1)

stop = instrument.add_component("stop", "Union_stop")

In [ ]:
instrument.show_diagram()

### Question 1
Add a logger that records scattering projected onto a 2D plane, the component is called "Union_logger_2D_space". Place it before the master and have it record the z x plane. It should look at an area of +/- 7 cm in each direction with 200 bins. Provide a reasonable filename and place it such that it views the center of the cryostat.

In [ ]:
# Your code here
logger_zx = instrument.add_component("logger_zx", "Union_logger_2D_space", before=master)
logger_zx.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=200,
                         D_direction_2='"x"', D2_min=-0.07, D2_max=0.07, n2=200,
                         filename='"logger_zx.dat"')
logger_zx.set_AT(0, cryostat_center)

In [ ]:
quiz.question_1(instrument)

### Question 2
Add a similar logger, but in the zy plane instead of zx. In ther vertical direction this should go from - 10 cm to 20 cm.

In [ ]:
logger_zy = instrument.add_component("logger_zy", "Union_logger_2D_space", before=master)
logger_zy.set_parameters(D_direction_1='"z"', D1_min=-0.07, D1_max=0.07, n1=200,
                       D_direction_2='"y"', D2_min=-0.1, D2_max=0.2, n2=200,
                       filename='"logger_zy.dat"')
logger_zy.set_AT(0, cryostat_center)

In [ ]:
quiz.question_2(instrument)

### Question 3
Add a Union_logger_2DQ component to investigate the scattering in the zx plane of the system. It should have limits +/- 3.3 Å$^{-1}$ and 200 bins in each direction.

In [ ]:
logger_2DQ = instrument.add_component("logger_2DQ", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min = -3.3, Q1_max = 3.3, n1 = 200,
                          Q_direction_2 = '"x"', Q2_min = -3.3, Q2_max = 3.3, n2 = 200,
                          filename = '"logger_2DQ_zx.dat"')

In [ ]:
quiz.question_3(instrument)

## Run the simulation
Time to run the simulation, but first we should consider at what settings.

Our YBaCuO sample has the 010 axis along the z axis and have 010 allowed with d = 3.8843. Here we calculate the necessary rotation of the crystal for satisfying the Bragg condition.

In [ ]:
%matplotlib widget
import mcstasscript.jb_interface as ms_widget

In [ ]:
import math
wavelength = 5.0
theta = 180/3.14159*math.asin(wavelength/2.0/3.8843)
print("theta = ", theta, " at wavelength = ", wavelength)

Use the simple math blog above to select appropriate parameters for the simulation. The log plot option with limited orders of magnitude really helps reveal detail in the plots.

In [ ]:
ms_widget.show(instrument)

### Question 4
Run a simulation with 1E7 rays at Bragg condition for the 010 reflection at a wavelength of 4 Å  and 5 Å. Look at the background visible on banana_tof_detector / banana_detector for both. Which of these have the most background?

- A: The simulation at $\lambda=4$Å
- B: The simulation at $\lambda=5$Å

In [ ]:
quiz.question_4("A")

### Question 5
Look at the logger results at the two wavelengths, what clue do they give of the big difference between the background at the two energies?

- A: Additional scattering from the Sample in the form of incoherent scattering
- B: Additional scattering from the Sample in the form of single crystal scattering
- C: Additional scattering from the Cryostat in the form of incoherent scattering
- D: Additional scattering from the Cryostat in the form of powder scattering

In [ ]:
quiz.question_5("D")

## Investigate deeper
Though the current loggers gave a lot of information about the system, we can investigate further. The next few questions will modify the current loggers and add additional to help clarify what is really happening in the system.

### Question 6
All our loggers contain all scattering orders. To view the different orders of scattering we can either add copies of all the loggers with the order_total parameter set or we can add a control parameter. Here we define a control parameter. It is your task to apply it to the three defined loggers in the system.

In [ ]:
order_total_par = instrument.add_parameter("order_total", value=0, comment="Scattering order recorded by loggers, 0 for all")

In [ ]:
logger_zx.order_total = order_total_par
logger_zy.order_total = "order_total" # Alternative allowed syntax
logger_2DQ.order_total = order_total_par

In [ ]:
quiz.question_6(instrument)

### Add more loggers
To confirm our suspicion we add additional loggers that record scattering from the sample and cryostat respectively. 

### Question 7
Lets examine the scattering in the sample and shell independently. First add a Union_logger_2DQ similar to the one added in question 3, but that targets the sample with target_geometry. Include the order_total parameter.

In [ ]:
logger_2DQ = instrument.add_component("logger_2DQ_sample", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min=-3.3, Q1_max=3.3, n1=200,
                          Q_direction_2 = '"x"', Q2_min=-3.3, Q2_max=3.3, n2=200,
                          filename = '"logger_2DQ_sample_zx.dat"',
                          order_total=order_total_par, target_geometry='"sample"')

In [ ]:
quiz.question_7(instrument)

### Question 8
Now add a similar logger that targets the two cryostat geometries. Include the order_total parameter.

In [ ]:
logger_2DQ = instrument.add_component("logger_2DQ_cryo", "Union_logger_2DQ", before=master)
logger_2DQ.set_parameters(Q_direction_1 = '"z"', Q1_min=-3.3, Q1_max=3.3, n1=200,
                          Q_direction_2 = '"x"', Q2_min=-3.3, Q2_max=3.3, n2=200,
                          filename = '"logger_2DQ_cryo_zx.dat"', order_total=order_total_par, 
                          target_geometry='"orange_layer_0_layer,orange_layer_1_layer"')

In [ ]:
quiz.question_8(instrument)

## Run the simulation with new loggers
Use the log plot option with limited orders of magnitude to see detail in the plots. If your system supports it, use mpi equal to the number of cores in your computer to speed up the simulation.

In [ ]:
wavelength = 5.0
theta = 180/3.14159*math.asin(wavelength/2.0/3.8843)
print("theta = ", theta, " at wavelength = ", wavelength)

In [ ]:
ms_widget.show(instrument)

### Question 9
Run the simulation with a wavelength of 4 Å and order_total=1. Look at the logger showing scattering in the cryostat. What are the two vertical lines?

- A: First order incoherent scattering in cryostat
- B: First order powder scattering in cryostat
- C: First order single crystal scattering in the sample
- D: Second order incoherent scattering in cryostat
- E: Second order powder scattering in cryostat
- F: Second order single crystal scattering in the sample

In [ ]:
quiz.question_9("B")

### Question 10
Run the simulation in the Bragg condition with a wavelength of 4 Å and the order_total parameter of 1 to see the first order of scattering. Find the Bragg peak on the 2D scattering vector logger (only a single pixel). Where is it located?

- A: $Q_z$=  0.49 Å$^{-1}$, $Q_x$ =  0.00 Å$^{-1}$
- B: $Q_z$= -0.24 Å$^{-1}$, $Q_x$ = -1.02 Å$^{-1}$
- C: $Q_z$=  0.88 Å$^{-1}$, $Q_x$ = -1.40 Å$^{-1}$
- D: $Q_z$=  1.21 Å$^{-1}$, $Q_x$ =  0.48 Å$^{-1}$

In [ ]:
quiz.question_10("C")

### Question 11
Run the simulation in the Bragg condition with a wavelength of 4 Å and the order_total parameter of 0 to see all  orders of scattering. On the 2D Q logger we 4 lines, two at constant $Q_z$ and two at an angle. We wish to understand the two at an angle. Use the parameters available and the loggers looking at only sample / cryostat to find the origin of the two lines by running additional simulations.

- A: First order scattering in sample
- B: First order scattering in cryostat
- C: Second order scattering in sample (2 scattering events in sample)
- D: Second order scattering in cryostat (2 scattering events in cryostat)
- E: Second order scattering in sample (first cryostat, then sample)
- F: Second order scattering in cryostat (first sample, then cryostat)

In [ ]:
quiz.question_11("F")

### Question 12
In the logger 2D Q recording scattering at all orders we see some broad lines across the narrow lines. Look at these at different values of order_total. What are these?

- A: Powder scattering from a ray that scattered from the sample at previous scattering
- B: Incoherent scattering from a ray that scattered from the sample at previous scattering
- C: Powder scattering from a ray that scattered from the cryostat at previous scattering
- D: Incoherent scattering from a ray that scattered from the cryostat at previous scattering

In [ ]:
quiz.question_12("C")